# MMQ para dados climáticos

In [8]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

In [9]:
#importar dados
df = pd.read_excel('Data/final_data.xlsx')

## CO2xTempo

In [10]:
#definir variáveis x e y
y = np.array(df['Co2 ppm'])
x = np.array(df['decimal date'])

### Linear

In [11]:
#Preparar model e treiná-lo, com teste t de student para significância

x_ = x
y_ = y
x_ = sm.add_constant(x_)
model = sm.OLS(y_,x_)
res = model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 3.338e+04
Date:                Thu, 15 Aug 2024   Prob (F-statistic):               0.00
Time:                        09:46:15   Log-Likelihood:                -2313.7
No. Observations:                 780   AIC:                             4631.
Df Residuals:                     778   BIC:                             4641.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2907.5485     17.881   -162.606      0.000   -2942.649   -2872.448
x1             1.6404      0.009    182.706      0.000       1.623       1.658
==============================================================================
Omnibus:                       43.349   Durbin-Watson:                   0.069
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.313
Skew:                           0.545   Prob(JB):                     2.39e-10
Kurtosis:                       2.580   Cond. No.                     2.11e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Logarítmico

In [12]:
#Preparar model e treiná-lo, com teste t de student para significância
x_ = np.log(x)
y_ = y

x_ = sm.add_constant(x_)
model = sm.OLS(y_,x_)
res = model.fit()
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.976
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                 3.178e+04
Date:                Thu, 15 Aug 2024   Prob (F-statistic):               0.00
Time:                        09:46:15   Log-Likelihood:                -2332.5
No. Observations:                 780   AIC:                             4669.
Df Residuals:                     778   BIC:                             4678.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.444e+04    139.130   -175.675      0.000   -2.47e+04   -2.42e+04
x1          3264.7262     18.315    178.257      0.000    3228.774    3300.678
==============================================================================
Omnibus:                       45.071   Durbin-Watson:                   0.066
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.080
Skew:                           0.555   Prob(JB):                     9.86e-11
Kurtosis:                       2.571   Cond. No.                     6.23e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Exponencial

In [13]:
#Preparar model e treiná-lo, com teste t de student para significância
x_ = x
y_ = np.log(y)

x_ = sm.add_constant(x_)
model = sm.OLS(y_,x_)
res = model.fit()
resSum = res.summary()

### Potencial

In [14]:
#Preparar model e treiná-lo, com teste t de student para significância
x_ = np.log(x)
y_ = np.log(y)

x_ = sm.add_constant(x_)
model = sm.OLS(y_,x_)
res = model.fit()
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                 4.998e+04
Date:                Thu, 15 Aug 2024   Prob (F-statistic):               0.00
Time:                        09:46:15   Log-Likelihood:                 2438.1
No. Observations:                 780   AIC:                            -4872.
Df Residuals:                     778   BIC:                            -4863.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -62.7609      0.307   -204.416      0.000     -63.364     -62.158
x1             9.0358      0.040    223.569      0.000       8.956       9.115
==============================================================================
Omnibus:                       27.655   Durbin-Watson:                   0.105
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.250
Skew:                           0.437   Prob(JB):                     7.34e-07
Kurtosis:                       2.677   Cond. No.                     6.23e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Polinomial

In [21]:
#Preparar model e treiná-lo, com teste t de student para significância
x_ = x
y_ = y

x_p = np.column_stack((x_, x_**2))


x_p = sm.add_constant(x_p)
model = sm.OLS(y_,x_p)
res = model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 7.540e+04
Date:                Thu, 15 Aug 2024   Prob (F-statistic):               0.00
Time:                        09:48:13   Log-Likelihood:                -1732.1
No. Observations:                 780   AIC:                             3470.
Df Residuals:                     777   BIC:                             3484.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.918e+04   1007.098     48.835      0.000    4.72e+04    5.12e+04
x1           -50.6753      1.011    -50.101      0.000     -52.661     -48.690
x2             0.0131      0.000     51.723      0.000       0.013       0.014
==============================================================================
Omnibus:                      113.576   Durbin-Watson:                   0.307
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.884
Skew:                          -0.092   Prob(JB):                     5.35e-07
Kurtosis:                       2.075   Cond. No.                     5.00e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  5e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Anomomalias de Temperatura x Tempo

In [10]:
#definir variáveis x e y
y = np.array(df['Co2 ppm'])
x = np.array(df['decimal date'])

# Anomomalias de Temperatura x Co2